# 뉴스기사 제목 및 링크 크롤링

네이버 부동산 상위 4개 뉴스데이터를 크롤링해 가져오는 코드입니다.


### 1. 배경

부동산 거래 내역과 함께, 현재 이슈가 되는 부동산 뉴스를 보면 거래 내역 파악 및 추이 파악에 도움이 될 것이라 생각했습니다. 

따라서 파이썬 FAST API 프레임워크를 통해 네이버부동산 최신 뉴스를 크롤링하여 가져옵니다.


### 2. 코드


In [14]:
import requests
from bs4 import BeautifulSoup

search = "https://land.naver.com/news/region.naver?city_no=1100000000&dvsn_no="

response = requests.get(search)
soup = BeautifulSoup(response.text, "html.parser")
dong_codes = soup.select(".area") # 결과는 리스트

# for code in dong_codes:
#     title = code.text
#     dong_code = code.attrs['href'].split("dvsn_no")[1][1:]
#     print(title, dong_code)

links = soup.select(".category_list li a:nth-of-type(2)") 

# for link in links:
#     title = link.text.strip()
#     url = link.attrs['href']
#     print(title, url)
response = {"news":[]}
for link, code in zip(links, dong_codes):
    city_name = code.text[1:-1]
    dong_code = code.attrs['href'].split("dvsn_no")[1][1:]
    url = link.attrs['href']

    title_soup = BeautifulSoup(requests.get(url).text, "html.parser")
    title = title_soup.select('#title_area span')[0].text
    ref = title_soup.select('.ofhd_float_title_text_press')[0].text
    if title_soup.select('#img1')!=[]:
        img = title_soup.select('#img1')[0].attrs['data-src']
    else :
        img = ""
    data = {"dong_code":dong_code, "title":title,"url":url,"city_name":city_name, "ref":ref, "img":img}
    response["news"].append(data)
    print(data)




{'dong_code': '1165000000', 'title': "성동구, 연탄 제로' 지원사업 시행", 'url': 'https://n.news.naver.com/article/011/0004417933', 'city_name': '서초구', 'ref': '서울경제', 'img': 'https://imgnews.pstatic.net/image/011/2024/11/21/0004417933_001_20241121134511458.jpg?type=w860'}
{'dong_code': '1111000000', 'title': '종로구, 내년 9월부터 버스비 지원', 'url': 'https://n.news.naver.com/article/011/0004417922', 'city_name': '종로구', 'ref': '서울경제', 'img': 'https://imgnews.pstatic.net/image/011/2024/11/21/0004417922_001_20241121134509081.jpg?type=w860'}
{'dong_code': '4143000000', 'title': '경기 월암지구 사전청약 당첨자 절반이 본청약 포기…분양가 상승 여파', 'url': 'https://n.news.naver.com/article/028/0002717138', 'city_name': '의왕시', 'ref': '한겨레', 'img': 'https://imgnews.pstatic.net/image/028/2024/11/20/0002717138_001_20241121020254996.jpg?type=w860'}
{'dong_code': '1126000000', 'title': '서울시, 상봉역 인근에 장기전세주택 223가구 공급한다', 'url': 'https://n.news.naver.com/article/015/0005059352', 'city_name': '중랑구', 'ref': '한국경제', 'img': 'https://imgnews.pstatic.net/image/

In [20]:
data

{'dong_code': '4159000000',
 'title': 'LH, 동탄2 ‘종합병원 건립 패키지형 개발사업’ 사업자공모 추진',
 'url': 'https://n.news.naver.com/article/119/0002894004',
 'city_name': '화성시',
 'ref': '데일리안',
 'img': 'https://imgnews.pstatic.net/image/119/2024/11/18/0002894004_001_20241118163212910.png?type=w860'}

## 2. Database 저장

local의 mysql db와 연결하여 크롤링한 기사를 입력하는 데이터합니다.

# 2. 뉴스기사 내용 요약 (미완)

In [44]:
search = "https://n.news.naver.com/article/009/0005398589"
response = requests.get(search)
soup = BeautifulSoup(response.text, "html.parser")
text = soup.select_one("#dic_area").get_text(strip=True)

print(text)

상권 개척했던 맛집 '효뜨'건물주와 갈등 끝 명도소송일대 임대료 3~4년 사이2배 껑충 뛰자 분쟁 잦아져유동인구 최근 줄어들며경리단길 전철 밟나 우려서울 용산구 한강대로에 위치한 베트남 음식점 효뜨. 효뜨서울의 맛집 '핫플레이스'로 유명한 용리단길이 최근 임대료 급등으로 곳곳에서 갈등이 불거지고 있는 것으로 나타났다. 용리단길 개척 주자 중 하나인 베트남 음식점 '효뜨'는 임대료 이견 끝에 건물주와 명도소송 중이다. 이런 현상이 계속 이어지면 용리단길 역시 앞서 젠트리피케이션으로 쇠락한 이태원 경리단길이나 신사동 가로수길과 같은 길을 걸을 것이란 전망도 나온다.18일 서울서부지방법원에 따르면 '효뜨'는 2019년 5월 용리단길에 입점해 월세 450만원(부가세 별도), 관리비 0원, 임차보증금 5000만원으로 임대차 계약을 했다. 임대 면적은 53.8㎡(약 16.3평)지만 지하주차장 30.81㎡와 당시 불법으로 증축됐던 위반 건축물 면적 30㎡를 점유해 사용할 수 있었다.효뜨는 신용산 상권을 만들었다고 평가받는 남준영 셰프의 첫 식당이다. GS25와 '효뜨 소고기 쌀국수' '효뜨 돼지고기 덮밥' 등 컬래버레이션 제품을 출시할 정도로 선풍적 인기를 끌었으며 강남점, 스타필드수원점 등 분점도 냈다.'효뜨'는 2년 뒤인 2021년 5월 월세 470만원, 관리비 50만원으로 임대료를 상향해 재계약을 했다. 그러나 이후 코로나19 시국인 2021년 7월부터 10월까지 월세를 연체하며 계약갱신청구권을 상실하게 됐다. 현행법에서는 3개월분 이상 월세가 연체되면 임대인은 임차인의 계약 갱신 요구를 거부할 권리를 갖게 된다.엎친 데 덮친 격으로 그 이듬해 불법 증축 건축물이 용산구청에 적발되며 원상복구 명령도 떨어졌다. 원상복구 후 재입점 조건을 협의하는 과정에서 갈등이 더 커진 것이다. 효뜨는 지난해 3월 주변 시세대로 권리금 3억원을 받고 계약을 종료하거나 1억400만원의 보상금을 받고 리뉴얼 후 재입점하는 2가지 안을 제시했다. 그러나 임대차 계약 기간이 종료된 지난해 6

In [ ]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

### 3. GPT 프롬프트 사용 (미완)

In [2]:
from openai import OpenAI
#key = ''
#client = OpenAI(api_key=key)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    response_format={ "type": "json_object" },
    messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "당신은 뛰어난 개발자입니다."},
    {"role": "user", "content": "매력적인 부동산 매물 거래내역 제공 서비스 캐치프레이즈를 만들어주세요"},
    ]
)

result = response.choices[0].message.content
print(result)

## https://hyunicecream.tistory.com/78 이거 참고해서 gpt 키 더 써보기

{"catchphrase": "당신의 꿈을 찾는 부동산, 거래의 매력을 전하다!"}


In [1]:
import subprocess

def list_installed_packages():
    result = subprocess.run(["pip", "list"], capture_output=True, text=True)
    print(result.stdout)

if __name__ == "__main__":
    list_installed_packages()

Package            Version
------------------ -----------
accelerate         1.1.1
annotated-types    0.7.0
anyio              4.6.2.post1
asttokens          2.4.1
asyncmy            0.2.9
beautifulsoup4     4.12.3
Brotli             1.0.9
certifi            2024.8.30
cffi               1.17.1
charset-normalizer 3.3.2
click              8.1.7
colorama           0.4.6
comm               0.2.2
cryptography       43.0.3
databases          0.9.0
debugpy            1.8.8
decorator          5.1.1
distro             1.9.0
exceptiongroup     1.2.2
executing          2.1.0
fastapi            0.115.5
filelock           3.13.1
fsspec             2024.10.0
gmpy2              2.1.2
greenlet           3.1.1
h11                0.14.0
httpcore           1.0.7
httpx              0.27.2
huggingface-hub    0.26.2
idna               3.7
importlib_metadata 8.5.0
ipykernel          6.29.5
ipython            8.18.1
jedi               0.19.2
Jinja2             3.1.4
jiter              0.7.1
jupyter_client    